![IPL Logo](ipl.png)
# IPL Wicket Predictor based on a Neural Network Architecture

* In this notebook I will demonstrate a fully connected deep neural network to predict whether a bastman will get out based on certain input constraints like innings, ball number, over, batsman, bowler.
* A practical application of this neural network would allow IPL teams to take leverage of the massive amounts of data collected during a match and use them to make adjustments in the team to fit the respective playing conditions.
* This neural network model uses four multi-layer perceptron layers with sixty neurons each to optimize performance. 

### Step 1 - Preprocessing Data for Network
* To provide valid label data to the network, I used the scikitlearn LabelEncoder and one hot ecoding method to prepare the data for the network.
* Categorical variables like batsmen and bowler are encoded using the scikitlearn LabelEncoder.

### Step 2 - Model definition
* As stated before I used a multi-layer perceptron and a fully connected neural network architecture to build a model.

### Step 3 - Training
* ```
learning_rate = 0.01
training_epochs = 100
layer1 = 60
layer2 = 60
layer3 = 60
layer4 = 60
```

[Data set](https://www.kaggle.com/manasgarg/ipl) used for this exercise

## Step 1 - Preprocessing

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def read_and_encode():
	df = pd.read_csv("~/.kaggle/datasets/manasgarg/ipl/deliveries.csv")
	innings = df[df.columns[1]].values
	over = df[df.columns[4]].values
	ball = df[df.columns[5]].values
	batsmen = encode_players(df[df.columns[6]].values)
	bowlers = encode_players(df[df.columns[8]].values)
	dismissed = df[df.columns[18]].values
	# encode dismissals to 0 or 1 for each row
	dismissed = encode_dismissals(len(dismissed), dismissed)
	X = np.column_stack((innings, over, ball, batsmen, bowlers))
	return (X, dismissed)

	
def encode_players(players):
	encoder = LabelEncoder()
	encoder.fit(players)
	return encoder.transform(players)


def encode_dismissals(rows, dismissals):
	r = np.zeros((rows, 2))
	for i, row in enumerate(dismissals):
		if row is not np.nan:
			r[i, 1] = 1
		else:
			r[i, 0] = 1
	return r

## Step 2 - Model Definition

In [2]:
X, Y = read_and_encode()

X, Y = shuffle(X, Y, random_state=10)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=415)

print(train_x.shape, train_y.shape)

# Important parameters for running the machine learning algorithm
learing_rate = 0.01
training_epochs = 100
cost_history = np.empty(shape=[1], dtype=float)
n_dim = X.shape[1]
print("n_dim", n_dim)
n_class = Y.shape[1]

#Define the number of hidden layers and the number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

x = tf.placeholder(tf.float32, [None, n_dim]) # this will be a matrix of n x X.shape[1]
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [None, n_class])

# Define the model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)

    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)

    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)

    #Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Define weights and biases for each layer
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, n_class]))
}

biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

(120368, 5) (120368, 2)
n_dim 5


## Step 3 - Training and Testing Accuracy

In [ ]:

# Initialize all variables
init = tf.global_variables_initializer()

saver = tf.train.Saver()


# Call your model defined
y = multilayer_perceptron(x, weights, biases)

# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learing_rate).minimize(cost_function)

sess = tf.Session()
sess.run(init)
model_path = os.getcwd() + "/models/ipl.ckpt"

# Calculate the cost and the accuracy for each epoch
mse_history = []
accuracy_history = []

print("Start training...")

for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={x: train_x, y_: train_y})
    cost = sess.run(cost_function, feed_dict={x: train_x, y_: train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    pred_y = sess.run(y, feed_dict={x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))

    print('epoch: ', epoch, ' - ', 'cost: ', cost, ' - MSE: ', mse_, \
        "- Train Accuracy: ", accuracy)

save_path = saver.save(sess, model_path, global_step=training_epochs)
print("Model saved in file: %s" % save_path)

# Print final accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_:test_y})))

# Print final MSE

pred_y = sess.run(y, feed_dict={x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))